In [ ]:
import time
time.gmtime()

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [3]:
import pandas as pd
import itertools, collections
import time
import numpy as np

from _code import DeepLogModel
from _code import generator
from _code import seed
from _code import trie

In [4]:
### Parameters
_seed=3
subdir = "dynamic-3"
vocabulary_size = 20
num_patterns = 10
num_new_patterns = 3

In [5]:
seed.seed(_seed)
subdir = subdir

In [6]:
vocabulary_size = vocabulary_size
num_patterns = num_patterns
vocabulary = generator.make_vocabulary(vocabulary_size=vocabulary_size)
patterns = generator.generate_patterns(num_patterns=num_patterns, vocabulary=vocabulary, min_pattern_size=3, max_pattern_size=7)
trie_g = trie.calc_g_value(patterns)
trie_h = trie.calc_h_value(patterns)
print("trie-g:", trie_g)
print("trie-h:", trie_h)
text_train, marks_train = generator.generate_text(patterns, text_size=50000, anomaly_ratio=0.00, vocabulary=vocabulary) 
tests1 = generator.generate_tests(patterns, vocabulary, n=700, text_size = 4, anomaly_ratio=0.1)

trie-g: 7
trie-h: 3


In [7]:
import json, os

def write_inputs1(subdir, vocabulary_size, vocabulary, patterns, text_train, marks_train, tests):
    dir_= os.path.join("..\\inputs", subdir)
    os.makedirs(dir_, exist_ok=True)
    j1={
        "vocabulary_size" : vocabulary_size,
        "vocabulary" : vocabulary,
        "workflows" : patterns
    }
    json.dump(j1, open(os.path.join(dir_, "v-wf.json"), "w"))

    j2 = {"text_train" : text_train, "marks_train" : marks_train}
    json.dump(j2, open(os.path.join(dir_, "train.json"), "w"))

    j3 = {"tests" : tests}
    json.dump(j3, open(os.path.join(dir_, "tests1.json"), "w"))

def read_inputs1(subdir):
    dir_ = os.path.join("..\\inputs", subdir)
    j1 = json.load(open(os.path.join(dir_, "v-wf.json")))
    j2 = json.load(open(os.path.join(dir_, "train.json")))
    j3 = json.load(open(os.path.join(dir_, "tests1.json")))
    
    vocabulary_size = j1["vocabulary_size"]
    vocabulary = j1["vocabulary"]
    patterns = j1["workflows"]
    patterns = [tuple(p) for p in patterns]
    
    text_train = j2["text_train"]
    marks_train = j2["marks_train"]
    
    tests_str_keys = j3["tests"]
    tests = dict()
    for k,v in tests_str_keys.items():
        tests[int(k)] = tuple(v)
    
    
    return vocabulary_size, vocabulary, patterns, text_train, marks_train, tests

write_inputs1(subdir, vocabulary_size, vocabulary, patterns, text_train, marks_train, tests1)

In [8]:
ret_vocabulary_size, ret_vocabulary, ret_patterns, ret_text_train, ret_marks_train, ret_tests1 = read_inputs1(subdir)

assert ret_vocabulary_size == vocabulary_size
assert ret_vocabulary == vocabulary
assert ret_patterns == patterns
assert ret_text_train == text_train
assert ret_marks_train == marks_train
assert ret_tests1 == tests1


In [9]:
deep_log_model = DeepLogModel.DeepLogModel(h=trie_h+1, n=vocabulary_size, vocabulary=vocabulary)
deep_log_model.build(num_lstm_layers=2, lstm_size=64)
deep_log_model.fit(text_train,epochs=1)

In [10]:
b = time.time()
entries = list()
for k, (text_test, text_marks, anomaly) in tests1.items():
    for g in range(0, vocabulary_size+1):
        res = deep_log_model.monitor_session(text_test, text_marks, g=g)
        entry = (k,g,res)
        entries.append(entry)
e = time.time()
print("time:", round(e-b,3), "seconds")

time: 706.402 seconds


In [11]:
df1 = pd.DataFrame(entries, columns = ["i","g","status"])
ddf1 = df1.pivot_table(index="i", columns = "g", values = "status", aggfunc = "sum")
def calc(s):
    c = dict(collections.Counter(s))
    TP = c.get("TP", 0)
    TN = c.get("TN", 0)
    FP = c.get("FP", 0)
    FN = c.get("FN", 0)
    eps = 1e-9
    
    prec = TP / (TP + FP + eps)
    rec = TP / (TP + FN + eps)
    acc = (TP + TN) / (TP + TN + FP + FN + eps)
    f1 = 2*(prec*rec)/(prec+rec+eps)
    
    return {"prec" : prec, "rec" : rec, "acc" : acc, "f1" : f1, "TP" : TP, "TN" : TN, "FP" : FP, "FN" : FN}
    
e=ddf1.apply(calc, axis=0)
e1=pd.DataFrame(list(e.values))

print("measure results for each g-value")
e1

measure results for each g-value


,prec,rec,acc,f1,TP,TN,FP,FN
0,0.250000,1.000000,0.250000,0.400000,175,0,525,0
1,0.250716,1.000000,0.252857,0.400916,175,2,523,0
2,0.272586,1.000000,0.332857,0.428397,175,58,467,0
3,0.323364,0.988571,0.480000,0.487324,173,163,362,2
4,0.360341,0.965714,0.562857,0.524845,169,225,300,6
5,0.422043,0.897143,0.667143,0.574040,157,310,215,18
6,0.565543,0.862857,0.800000,0.683258,151,409,116,24
7,1.000000,0.805714,0.951429,0.892405,141,525,0,34
8,1.000000,0.737143,0.934286,0.848684,129,525,0,46
9,1.000000,0.702857,0.925714,0.825503,123,525,0,52


In [12]:
### finding emprical best g
display(e1.loc[[e1["acc"].argmax()]])
display(e1.loc[[e1["f1"].argmax()]])

,prec,rec,acc,f1,TP,TN,FP,FN
7,1.0,0.805714,0.951429,0.892405,141,525,0,34


,prec,rec,acc,f1,TP,TN,FP,FN
7,1.0,0.805714,0.951429,0.892405,141,525,0,34


In [13]:
best_g = e1["f1"].argmax()
print("best-g", best_g)

best-g 7


## online_cases

In [14]:
num_new_patterns = num_new_patterns
new_patterns = generator.generate_patterns(num_patterns=num_new_patterns, vocabulary=vocabulary, min_pattern_size=3, max_pattern_size=7)

online_cases = generator.generate_tests(patterns + new_patterns, vocabulary, n=10000, text_size = 4, anomaly_ratio=0.0)    

In [15]:
def write_inputs2(subdir, online_cases):
    dir_= os.path.join("..\\inputs", subdir)
    os.makedirs(dir_, exist_ok=True)
    
    j4 = {"online_cases" : online_cases}
    json.dump(j4, open(os.path.join(dir_, "online_cases.json"), "w"))

def read_inputs2(subdir):
    dir_ = os.path.join("..\\inputs", subdir)
    
    j4 = json.load(open(os.path.join(dir_, "online_cases.json")))
    online_cases_str_keys = j4["online_cases"]
    online_cases = dict()
    for k,v in online_cases_str_keys.items():
        online_cases[int(k)] = tuple(v)

    return online_cases

In [16]:
write_inputs2(subdir, online_cases)
ret_online_cases = read_inputs2(subdir)
assert ret_online_cases == online_cases

In [17]:
trie_g_new = trie.calc_g_value(patterns + new_patterns)
trie_h_new = trie.calc_h_value(patterns + new_patterns)
print("best-g:", best_g)
print("trie-g new: {}. (prev trie-g: {})".format(trie_g_new, trie_g))
print("trie-h new: {}. (prev trie-h: {})".format(trie_h_new, trie_h))
best_g_new = max(best_g, trie_g_new)
print("best-g-new:", best_g_new)

best-g: 7
trie-g new: 10. (prev trie-g: 7)
trie-h new: 3. (prev trie-h: 3)
best-g-new: 10


In [18]:
num_feedbacks = 0
num_feedbacks_new = 0
num_feedbacks_old = 0

def contains_new_pattern(text, new_patterns):
    new_ps_str = ["#".join(p) for p in new_patterns]
    text_str = "#".join(text)
    for new_p_str in new_ps_str:
        if new_p_str in text_str:
            return True
    return False
    
for index, case in online_cases.items():
    text, marks, anomaly_exists = case
    res = deep_log_model.monitor_session(text, marks, g=best_g)
    if res == "FP":
        deep_log_model.train_feedback(text, marks, g = best_g) # we are using here the old empirical best g. The theoretical g (trie_g_new) might be higher and give at the end worse results.
        num_feedbacks += 1
        if contains_new_pattern(text, new_patterns):
            num_feedbacks_new +=1
        else:
            num_feedbacks_old +=1 # count only totally "old" feedback cases without any new workflow 
        

In [19]:
print("num_feedbacks_new", num_feedbacks_new)
print("num_feedbacks_old", num_feedbacks_old)
print("num_feedbacks", num_feedbacks)

num_feedbacks_new 3698
num_feedbacks_old 1414
num_feedbacks 5112


In [20]:
tests2 = generator.generate_tests(patterns + new_patterns, vocabulary, n=700, text_size = 4, anomaly_ratio=0.1)

In [21]:
def write_inputs3(subdir, tests):
    dir_= os.path.join("..\\inputs", subdir)
    os.makedirs(dir_, exist_ok=True)
    
    j5 = {"tests" : tests}
    json.dump(j5, open(os.path.join(dir_, "tests2.json"), "w"))

def read_inputs3(subdir):
    dir_= os.path.join("..\\inputs", subdir)
    j5 = json.load(open(os.path.join(dir_, "tests2.json")))
    
    tests_str_keys = j5["tests"]
    tests = dict()
    for k,v in tests_str_keys.items():
        tests[int(k)] = tuple(v)
    return tests

In [22]:
write_inputs3(subdir, tests2)
ret_tests2 = read_inputs3(subdir)
assert ret_tests2 == tests2

In [23]:
b = time.time()
entries = list()
for k, (text_test, text_marks, anomaly) in tests2.items():
    for g in range(best_g, vocabulary_size+1): # adding WF can't cause lower g.
        res = deep_log_model.monitor_session(text_test, text_marks, g=g)
        entry = (k,g,res)
        entries.append(entry)
e = time.time()
print("time:", round(e-b,3), "seconds")

time: 475.995 seconds


In [24]:
df2 = pd.DataFrame(entries, columns = ["i","g","status"])
ddf2 = df2.pivot_table(index="i", columns = "g", values = "status", aggfunc = "sum")
def calc(s):
    c = dict(collections.Counter(s))
    TP = c.get("TP", 0)
    TN = c.get("TN", 0)
    FP = c.get("FP", 0)
    FN = c.get("FN", 0)
    eps = 1e-9
    
    prec = TP / (TP + FP + eps)
    rec = TP / (TP + FN + eps)
    acc = (TP + TN) / (TP + TN + FP + FN + eps)
    f1 = 2*(prec*rec)/(prec+rec+eps)
    
    return {"g" : s.name, "prec" : prec, "rec" : rec, "acc" : acc, "f1" : f1, "TP" : TP, "TN" : TN, "FP" : FP, "FN" : FN}
    
e=ddf2.apply(calc, axis=0)
e2=pd.DataFrame(list(e.values))

print("measure results for each g-value")
e2

measure results for each g-value


,g,prec,rec,acc,f1,TP,TN,FP,FN
0,7,0.433260,0.970588,0.621429,0.599092,198,237,259,6
1,8,0.498701,0.941176,0.707143,0.651952,192,303,193,12
2,9,0.638889,0.901961,0.822857,0.747967,184,392,104,20
3,10,1.000000,0.843137,0.954286,0.914894,172,496,0,32
4,11,1.000000,0.779412,0.935714,0.876033,159,496,0,45
5,12,1.000000,0.681373,0.907143,0.810496,139,496,0,65
6,13,1.000000,0.602941,0.884286,0.752294,123,496,0,81
7,14,1.000000,0.529412,0.862857,0.692308,108,496,0,96
8,15,1.000000,0.475490,0.847143,0.644518,97,496,0,107
9,16,1.000000,0.372549,0.817143,0.542857,76,496,0,128


In [25]:
best_result2 = e2.loc[[e2["f1"].argmax()]]
best_g2 = best_result2["g"].iloc[0]

In [26]:
e3 = e2[e2["g"] == best_g2].copy()
e3["num_feedbacks_new"] = num_feedbacks_new
e3["num_feedbacks_old"] = num_feedbacks_old
e3["num_feedbacks"] = num_feedbacks
e3

,g,prec,rec,acc,f1,TP,TN,FP,FN,num_feedbacks_new,num_feedbacks_old,num_feedbacks
3,10,1.0,0.843137,0.954286,0.914894,172,496,0,32,3698,1414,5112
